#1.GPT - Generative Pre-trained Transformer
- https://wikidocs.net/186266

##1-1.설정

In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import random
from tqdm import tqdm
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

In [ ]:
model=TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)  # 두 개의 문장이 이어지는 문장 관계인지 여부를 판단하는 GPT 구조를 로드
tokenizer=AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')    # 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.10.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'lm_head.weight', 'transformer.h.6.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias',

##1-2. KoGPT-2로 문장 생성하기

In [ ]:
sent='근육이 커지기 위해서는'

In [ ]:
input_ids=tokenizer.encode(sent)
input_ids=tf.convert_to_tensor([input_ids])
print(input_ids)    #  1) 5개의 정수 시퀀스

tf.Tensor([[33245 10114 12748 11357]], shape=(1, 4), dtype=int32)


In [ ]:
# 2) 정수 시퀀스를 GPT의 입력으로 사용하여 GPT가 이어서 문장을 생성
output=model.generate(input_ids,
                      max_length=128,
                      repetition_penalty=2.0,  # 문장 생성 시 동일한 단어나 구가 반복되는 것을 방지 1.0 ~ 2.0+
                      use_cache=True)          # 계산 결과를 캐시에 저장 >> 생성 속도를 빠르게 하기 위한 옵션

output_ids=output.numpy().tolist()[0]
print(output_ids)

[33245, 10114, 12748, 11357, 23879, 39306, 9684, 7884, 10211, 15177, 26421, 387, 17339, 7889, 9908, 15768, 6903, 15386, 8146, 12923, 9228, 18651, 42600, 9564, 17764, 9033, 9199, 14441, 7335, 8704, 12557, 32030, 9510, 18595, 9025, 10571, 25741, 10599, 13229, 9508, 7965, 8425, 33102, 9122, 21240, 9801, 32106, 13579, 12442, 13235, 19430, 8022, 12972, 9566, 11178, 9554, 24873, 7198, 9391, 12486, 8711, 9346, 7071, 36736, 9693, 12006, 9038, 10279, 36122, 9960, 8405, 10826, 18988, 25998, 9292, 7671, 9465, 7489, 9277, 10137, 9677, 9248, 9912, 12834, 11488, 13417, 7407, 8428, 8137, 9430, 14222, 11356, 10061, 9885, 19265, 9377, 20305, 7991, 9178, 9648, 9133, 10021, 10138, 30315, 21833, 9362, 9301, 9685, 11584, 9447, 42129, 10124, 7532, 17932, 47123, 37544, 9355, 15632, 9124, 10536, 13530, 12204, 9184, 36152, 9673, 9788, 9029, 11764]


In [ ]:
tokenizer.decode(output_ids)

'근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.\n특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.\n또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.\n아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.\n운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.\n운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.\n운동을'

In [ ]:
# Numpy로 Top 5 뽑기
output=model(input_ids)
print(output.logits.shape)  # 1개 문장, 길이 4, 단어 사전 크기 30522

top5=tf.math.top_k(output.logits[0, -1], k=5)  # [0 ~ 마지막]

(1, 4, 51200)


In [ ]:
tokenizer.convert_ids_to_tokens(top5.indices.numpy())    # 그 후 Top 5의 단어를 한국어로 변환하여 출력

['▁무엇보다', '▁우선', '▁반드시', '▁피부', '▁무엇보다도']

In [ ]:
# Numpy Top 5로 문장 생성하기
sent='근육이 커지기 위해서는'
input_ids=tokenizer.encode(sent)
print(input_ids)

while len(input_ids) < 50:
    output=model(np.array([input_ids]))

    # Top 5의 단어들을 추출
    top5 = tf.math.top_k(output.logits[0, -1], k=5)

    # Top 5의 단어들 중 랜덤으로 다음 단어로 선택.
    token_id = random.choice(top5.indices.numpy())
    input_ids.append(token_id)

tokenizer.decode(input_ids)

[33245, 10114, 12748, 11357]


'근육이 커지기 위해서는 무엇보다 면역력을 키우는 노력이 필요하다.\n또 평소 스트레스와 피로가 쌓여있는 직장인들을 위해서는 충분한 수분을 충분히 챙기고 수분 보충에 도움이 되어야 하며, 평소보다 수분의 함량이 적은 채소도 많이 먹는 것이 좋겠지만 수분의 함량이'

##1-3.한국어 챗봇

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import urllib.request

In [ ]:
# 두 개의 문장이 이어지는 문장 관계인지 여부를 판단하는 GPT 구조를 로드
tokenizer=AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')

# 해당 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드
model=TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.10.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'lm_head.weight', 'transformer.h.6.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.3.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [ ]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)
print('-' * 10)
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))

# </s> 1 : 시작,끝  / <pad> 3
# </s><usr> 12시 땡!<sys> 하루가 또 가네요.</s>

1
1
3
----------
</s>
<usr>
<pad>
<sys>


In [ ]:
# 챗봇 데이터를 로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data=pd.read_csv('ChatBotData.csv')
train_data.shape

(11823, 3)

In [ ]:
# 챗봇 전처리
def get_chat_data():
  for question, answer in zip(train_data.Q.to_list(), train_data.A.to_list()):
    bos_token=[tokenizer.bos_token_id]
    eos_token=[tokenizer.eos_token_id]
    sent=tokenizer.encode('<usr>' + question + '<sys>' + answer)   # 질문과 답변 전처리
    yield bos_token + sent + eos_token

In [ ]:
# batch_size=32   # GPU 메모리가 부족 16 8 4
batch_size=16
dataset=tf.data.Dataset.from_generator(get_chat_data, output_types=tf.int32)

In [ ]:
# 배치 크기만큼 데이터를 구성하되 패딩 토큰으로 패딩을 진행.
dataset=dataset.padded_batch(batch_size=batch_size,
                             padded_shapes=(None,),     # 각 시퀀스의 길이가 가변적, 가장 긴 시퀀스를 기준으로 나머지 시퀀스에 패딩을 추가
                             padding_values=tokenizer.pad_token_id)  # 3(패딩)

In [ ]:
# 첫번째 배치(첫 32개의 데이터 묶음)을 출력해 토큰, 1(시작,끝), 3(패딩)
for batch in dataset:
    print(batch)
    break

tf.Tensor(
[[    1     2  9349  7888   739  7318   376     4 12557  6824  9108  9028
   7098 25856     1     3     3     3     3     3     3]
 [    1     2  9020  8263  7497 10192 11615  8210  8006     4 12422  8711
   9535  7483 12521     1     3     3     3     3     3]
 [    1     2  9085  7597   395  8149 10624  7397 24224 13358  7182     4
  12079  8135 16899  9677  8234   389     1     3     3]
 [    1     2  9085  7597   395  8149  9465 10624  7397 24224 13358  7182
      4 12079  8135 16899  9677  8234   389     1     3]
 [    1     2  9943   422   418  9327  8702  7098     4  9847 16912 18328
   8671  7415  8263  8234   389     1     3     3     3]
 [    1     2  9815   410 21249 10174  6824  8210  8006     4  9427 11056
  11594 10137 10556  9266  8711 25856     1     3     3]
 [    1     2  9815   410 21249  9183  7249     4  9427 11056 11594 10137
  10556  9266  8711 25856     1     3     3     3     3]
 [    1     2  9815 37655  9622  8619 10401  9183  9328   216     4  944

In [ ]:
print(tokenizer.decode(batch[0]))
print("batch shape:", batch.shape)
print("batch dtype:", batch.dtype)

</s><usr> 12시 땡!<sys> 하루가 또 가네요.</s><pad><pad><pad><pad><pad><pad>
batch shape: (16, 21)
batch dtype: <dtype: 'int32'>


In [ ]:
# 옵티마이저 결정
adam=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

# 전체 데이터의 개수를 배치 크기로 나누면 하나의 에포크에서 실행되는 학습 횟수가 계산됨.
steps=len(train_data) // batch_size + 1

EPOCHS=1   # 3, batch=32

for epoch in range(EPOCHS):
  epoch_loss=0   # 한 epoch 동안의 누적 손실

  for batch in tqdm(dataset, total=steps):   # 배치 단위로 데이터셋 순회
      with tf.GradientTape() as tape:        # 자동 미분 준비
          result=model(batch, labels=batch)  # 모델 예측 (입력과 정답이 같음: autoencoder 또는 language model)
          loss=result[0]   # 손실 값 추출
          batch_loss=tf.reduce_mean(loss)  # 손실 평균값 계산

      grads = tape.gradient(batch_loss, model.trainable_variables)   # 손실에 대한 가중치의 gradient 계산
      adam.apply_gradients(zip(grads, model.trainable_variables))    # 옵티마이저로 가중치 업데이트
      epoch_loss += batch_loss / steps  # 평균 손실 누적

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

  0%|          | 0/739 [00:00<?, ?it/s]

[Epoch:    1] cost = 2.23705411


In [ ]:
# 챗봇 실행
text='오늘도 좋은 하루!'
sent='<usr>' + text + '<sys>'

input_ids=[tokenizer.bos_token_id] + tokenizer.encode(sent)
input_ids=tf.convert_to_tensor([input_ids])
print('정수 인코딩 후 :', input_ids)
print('정수 인코딩을 재복원 :', tokenizer.decode(input_ids[0]))

정수 인코딩 후 : tf.Tensor([[    1     2 10070  7235 10586 12557   376     4]], shape=(1, 8), dtype=int32)
정수 인코딩을 재복원 : </s><usr> 오늘도 좋은 하루!<sys>


In [ ]:
output=model.generate(input_ids, max_length=50, early_stopping=True, eos_token_id=tokenizer.eos_token_id)
decoded_sentence=tokenizer.decode(output[0].numpy().tolist())
print(decoded_sentence)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


</s><usr> 오늘도 좋은 하루!<sys> 좋은 하루를 만들어보세요.</s>


In [ ]:
output=model.generate(input_ids, max_length=50, do_sample=True, top_k=10)
decoded_sentence = tokenizer.decode(output[0].numpy().tolist())
print(decoded_sentence.split('<sys> ')[1].replace('</s>', ''))

좋은 하루를 만들 수 있죠.


In [ ]:
def return_answer_by_chatbot(user_text):
  sent='<usr>' + user_text + '<sys>'
  input_ids=[tokenizer.bos_token_id] + tokenizer.encode(sent)
  input_ids=tf.convert_to_tensor([input_ids])
  output=model.generate(input_ids, max_length=50, do_sample=True, top_k=20)
  sentence=tokenizer.decode(output[0].numpy().tolist())
  chatbot_response = sentence.split('<sys> ')[1].replace('</s>', '')
  return chatbot_response

In [ ]:
return_answer_by_chatbot('안녕! 반가워~')

'감사합니다.'

In [ ]:
return_answer_by_chatbot('너는 누구야?')

'제가 먼저 연락하고 저한테 물어보세요.'

In [ ]:
return_answer_by_chatbot('영화 해리포터 재밌어?')

'영화 한편 보는게 좋을 것 같아요.'

In [ ]:
return_answer_by_chatbot('너 딥 러닝 잘해?')

'직접 해보세요.'